# Getting started with Pandas TimeSeries

This notebook is intended to introduce you to the basic Pandas DateTime    
The following five points will be covered:

1. [Parsing DateTime](#task1)
2. [Aggregating columns](#task2)
3. [Extracting DateTime properties](#task3)
4. [Fitering and Selecting specific durations](#task4)
5. [Changing the granularity of the Timeseries](#task5)


### Prepare environment and read data 

In [ ]:
# Constants 
INPUT_PATH = '/kaggle/input/netflix-shows/netflix_titles.csv'

# Libraries 
import pandas as pd 
import matplotlib.pyplot as plt

# Set default properties for plotting 
plt.rcParams['figure.figsize'] = [11, 4]
plt.rcParams['figure.dpi'] = 100 

In [ ]:
# Read data and display 5 random entries 
raw_df = pd.read_csv(INPUT_PATH)
raw_df.sample(5)

_____

## Task 1: Countthe number of shows added per day     

In the following section, we will parse the raw date format into      
pandas datetime and summarize the daily shows added to the total number 

### Parse timestamp into datetime column <a id='task1'></a>

Change the raw format to a pandas datetime format.    
Once we have changed the format as such, we will be able to    
apply more functionalities illustrated below 


In [ ]:
# Copy original data
df = raw_df.copy()

# Parse the raw `date_added` column to pandas datetime format
df['date_added'] = pd.to_datetime(df['date_added'])

# Check the type of the `date_added` column
df['date_added'].dtypes

### Count shows added  per date <a id='task2'></a>
All the shows have been listed in the original dataframe.     
Now let's count the total number of shows added per day

In [ ]:
# For each date, count the number of show added
# Note the regular usegae of `groupby` method
shows_added = df.groupby('date_added')[['show_id']].count()

# Rename column to describe the new content 
shows_added = shows_added.rename({'show_id': 
                                  'number_of_shows_added'}, 
                                 axis=1)

# View and plot TimeSeries 
shows_added.plot()
shows_added

______

## Task 2: Extract the day name and sum-up the shows added 
<a id='task3'></a>

In the last step, we have used the `date_added` column to count the number of shows.    
Since we've used the `groupby` functionality to count the number of shows,     
the column is set as our index. 

We could now use our new index directly to extract the Attributes of the timestamp.    
One example of those Attributes is the `day_name`.    
Check out the [full list of the attributes here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html).

In [ ]:
# Add day name as a seperate column
shows_added['day_name'] = shows_added.index.day_name()

# Group by the extracted day name and sum up the 
total_per_weekday = shows_added.groupby('day_name')['number_of_shows_added'].sum()

# View results
# ... Seems that Netflix prefers to release the new shows on Fridays 
total_per_weekday

______

## Task 3: Select data from 2016 onwards 

You can also use the regular masking way to select and filter entries.      
The syntax is even simpler than one could expect. You don't even need to parse    
your filtering criteria to `datetime`. A simple string with `%YYYY-%MM-%DD` format     
will do the job  


In [ ]:
# Create mask of booleans from the required start date
# Note that the date is simple string and does not have to be a parsed datetime variable
mask = shows_added.index >= '2016-01-01'

# Apply the boolean mask to the dataframe 
shows_added = shows_added.loc[mask,:].copy()

# Plot the filtered data 
shows_added['number_of_shows_added'].plot()

______

## Task 4: Sum up weekly data 

It is possible to change the granularity of your timeseries directly using Pandas datetie module.        
       
       
To do that, you need to specify two things: 
- Your new granularity passed as an argument to the `resample` function. [Read more details](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)
- The function that will be used to generate the new granularity

In [ ]:
# Use the resampling function to group data per week
weekly_data = (shows_added.
               resample('1W')   # For each week 
               .sum())          # Calculate the sum

In [ ]:
# View and plot weekly data  
weekly_data.plot()
weekly_data